Cameron Jester
UML HW 2 PT 4

- Done -



PROBLEM 4 EM for coin flips

Three coins A,B,C with head-prob p_A, p_B, p_C can be chosen for each of N sessions. Once a coin is chosen for the session, that coin is flipped D times.
For D=20 and N=50, and fixed non-uniform selection coin probabilities pi_A, pi_B, pi_C, which sum to 1, we have this outcome , each row corresponds to a session with 20 binary 1=head 0=tail.

Compute the probabilities to select each coin to session (3 mixture "pi" probabilities), and also the bias probabilities (3 param "p" probabilities).

HINT: for each session, since the D flips are independent of each other, what matters is the number of heads out of the batch size D. If chance of head is p_ for each flip, then probability of observing x heads is binomial(x|p_, D). Here is a technical brief note .
In English: binomial(X|p_,D) = probability to get x heads out of D coin flips, if coin has head-bias p_.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = '/content/drive/My Drive/archive-3/coin_flips_outcome.txt'

In [ ]:
import random
import pandas as pd
import numpy as np
from scipy.stats import binom


data = pd.read_csv(filepath, header=None)
#need to do some preprocessing to get 1/0s in an array

data_array = []
for row in data.values:
    i_data = [list(map(int, entry.split())) for entry in row] #splitting by spaces since no comma, not strings so need to make #
    data_array.append(i_data)

data = np.array(data_array)


d = 20  #flips per session
n = 50  #sessions


head_counts = np.array([np.sum(row) for row in data])#getting the count of heads for each row since that will reprsent one session
 #can get the bias of coins from this later)


#met with a TA, who told me to select my own numbers to start this off
coins= 3
#p = np.random.rand(3)
p = np.array([0.70, 0.25, 0.62])
pi = np.ones(coins) / coins  ##just uniform 1/3 of selecting each coin to start,


def e_step(head_counts, d, p, pi):


    responsibilities = np.array([
        binom.pmf(head_counts, d, p[k])   #pmf = Probability Mass Function, gives the probability of # of heads in a given session given the bias assigned to coin
        #just looping through all three coins
        for k in range(coins)])
    responsibilities /= responsibilities.sum(axis=0)   #normalizing over all of the components for each coin, like would with cluster
    return responsibilities


def m_step(responsibilities, head_counts, d):
    #updating selection probs, this forumla comes from the site listed
    pi = responsibilities.sum(axis=1) / n

    #updating head bias,  this forumla comes from the site listed
    p = np.array([
    np.sum(responsibilities[j] * head_counts) / (d * np.sum(responsibilities[j]))
        for j in range(3)])



    log_likelihood = np.sum(np.log(np.sum([
        pi[k] * binom.pmf(head_counts, d, p[k]) for k in range(coins)
    ], axis=0)))

    return pi, p, log_likelihood

#actually running here, was told by a TA that 100 would be good
max_iter = 100
log_likelihoods = []

for iteration in range(max_iter):
    responsibilities = e_step(head_counts, d, p, pi)
    pi, p, log_likelihood = m_step(responsibilities, head_counts, d)


    log_likelihoods.append(log_likelihood)


print(p)
print(pi)
print(log_likelihoods)

[0.92663702 0.24910494 0.61699181]
[0.18495001 0.32961409 0.4854359 ]
[-153.42975141847927, -149.4097179773084, -147.15763573024174, -145.5401530224987, -144.2444709729008, -143.2426512823878, -142.58544575378062, -142.25399633469812, -142.12792501998956, -142.0890099998133, -142.0784637767738, -142.0760242265501, -142.07569315481155, -142.07580919493338, -142.07595603786572, -142.07605671035748, -142.07611490420703, -142.07614598118607, -142.07616185368346, -142.07616973858498, -142.0761735841533, -142.0761754360484, -142.07617631987583, -142.07617673895575, -142.07617693672506, -142.07617702972678, -142.07617707334643, -142.0761770937646, -142.07617710330823, -142.07617710776395, -142.07617710984243, -142.07617711081144, -142.07617711126298, -142.07617711147324, -142.07617711157118, -142.0761771116168, -142.07617711163797, -142.07617711164784, -142.0761771116525, -142.07617711165466, -142.07617711165562, -142.07617711165608, -142.07617711165628, -142.07617711165636, -142.076177111656


1. Coin 1:
   - p ≈ 0.237 (24% heads)
   - π ≈ 0.307 (31% of sessions)

2. Coin 2:
   - p ≈ 0.610 (61% heads)
   - π ≈ 0.515 (51% of sessions)

3. Coin 3:
   - p ≈ 0.932 (93% heads)
   - π ≈ 0.179 (18% of sessions)